### Figure 2 : Ripple oscillations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from subjects import sns_boxplot_kw,stat_kw, sns_violin_kw
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats
from statannotations.Annotator import Annotator
from plotters import violinplot


grpdata = subjects.GroupData()
group = ['NSD','SD']

# ripple
psd = grpdata.ripple_psd
examples = grpdata.ripple_examples

rpl_features = grpdata.ripple_features
ripple_autocorr = grpdata.ripple_autocorr
# ripple_autocorr = ripple_autocorr[ripple_autocorr.lag_time>=-1]

rate = grpdata.ripple_rate
pbe_rate = grpdata.pbe_rate

# frate
frate_in_rpl = grpdata.frate_in_ripple


plt.clf()
fig = Fig(grid=(8, 11), fontsize=7)
filename = subjects.figpath_sd / "figure2"

### Peak frequency and power

In [ ]:
# power['zscore'] = np.log10(power['zscore'])
y_labels = ["Ripple frequency (Hz)", "Ripple power (zscore)", "Sharp wave amplitude"]
for d, y in enumerate(["peak_freq", "ripple_power", "sw_amp"]):
    ax = fig.subplot(fig.gs[1, 2 * d : 2 * d + 2])
    plot_kw = dict(data=rpl_features, x="zt", y=y, hue="grp", hue_order=group, ax=ax)

    # sns.boxplot(**plot_kw, width=0.5, **sns_boxplot_kw)
    violinplot(**plot_kw)
    ax.tick_params("x", labelrotation=30)
    ax.legend([], frameon=False)
    ax.set_ylabel(y_labels[d])
    ax.set_xlabel("")

    # stats
    # stat_kw['loc'] = 'inside'
    orders = rpl_features.zt.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
    pairs = pairs + [(("0-2.5", "NSD"), ("5-7.5", "SD"))]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw, color="#4AB33E")
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    pairs = [(("2.5-5", "SD"), ("5-7.5", "SD"))]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw, color=subjects.colors_sd(1)[1])
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    pairs = [(("2.5-5", "NSD"), ("5-7.5", "NSD"))]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw, color=subjects.colors_sd(1)[0])
    annotator.apply_and_annotate()
    annotator.reset_configuration()


### Ripple autocorr
ax = fig.subplot(fig.gs[7:9])
sns.lineplot(
    data=ripple_autocorr,
    x="lag_time",
    y="auto_corr",
    hue="grp",
    hue_order=["NSD", "SD"],
    ci=None,
    palette=subjects.colors_sd(1),
    lw=1,
)
ax.legend("", frameon=False)
# ax.axis("off")


### Ripple and PBE rate

In [ ]:
y_labels = ["Ripple rate (Hz)", "PBE rate (Hz)"]
for d, (df, y) in enumerate(zip([rate, pbe_rate], ["rate", "rate"])):
    ax = fig.subplot(fig.gs[3, 2 * d : 2 * d + 2])
    plot_kw = dict(
        data=df,
        x="zt",
        y=y,
        hue="grp",
        hue_order=group,
        ax=ax,
        dodge=True,
    )

    sns.stripplot(
        **plot_kw, palette=subjects.colors_sd(1), edgecolor="w", linewidth=0.3, size=3
    )
    # violinplot(**plot_kw)
    ax.tick_params("x", labelrotation=30)
    ax.legend([], frameon=False)
    ax.set_ylabel(y_labels[d])
    ax.set_xlabel("")

    # stats
    # Across groups
    orders = df.zt.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw, color="#4AB33E")
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    # Within groups
    for i, g in enumerate(group):
        pairs2 = [((orders[_], g), (orders[_ + 1], g)) for _ in range(3)]
        annotator = Annotator(pairs=pairs2, **plot_kw, order=orders)
        annotator.configure(
            test="t-test_paired", **stat_kw, color=subjects.colors_sd(1)[i]
        )
        annotator.apply_and_annotate()
        annotator.reset_configuration()


### Firing inside/outside ripples

### saving

In [ ]:
fig.savefig(filename)